# Datenanalyse

Hier werden die im vorhergehenden Notebook vorbereiteten Daten analysiert.
Berücksichtigt wird nur der Standardfall für Museumsdaten, d.h. Datensätze im LIDO-XML-Format.

Die Analyse erfolgt mittels einer Klasse `LIDO`, die die Normdatenabdeckung eines Datensatzes analysiert und für festgelegte Kategorien einen Abdeckungsgrad zwischen 0 und 1 zurückgibt. Der Abdeckungsgrad ist `None`, wenn der Datensatz kein entsprechendes LIDO-Feld aufweist, eine Abdeckung also nicht berechnet werden kann.

Die Tabelle gibt einen Überblick über die zurückgegebenen Werte der Normdatenabdeckung:

| Attribut | ausgewertetes LIDO-Feld | Entsprechung im [Minimaldatensatz](www.minimaldatensatz.de) |
|:---------|:--------------|:-----------------|
| `.actorCoverage` | `<lido:actor/>` | Person/Körperschaft |
| `.placeCoverage` | `<lido:place/>` | Ort |
| `.subjectCoverage` | `<lido:subjectConcept/>` | Inhaltsschlagwort |
|`.objectWorkTypeCoverage` | `<lido:objectWorkType/>` | Objekttyp |
| `.cumulativeCoverage` | n/a | keine Entsprechung. Gibt den Mittelwert der oben aufgeführten Abdeckungsgrade bei Normdaten an. |

Neben der Normdatenabdeckung wird für Datums- und Maßangaben ein Standardisierungsgrad ermittelt:

| Attribut | ausgewertetes LIDO-Feld | Beschreibung |
|:---------|:--------------|:-----------------|
| `.dateCoverage` | `<lido:eventDate/>` | Anteil maschinenlesbarer Datumsangaben |
| `.measureCoverage` | `<lido:objectMeasurementsSet/>` | Anteil maschinenlesbarer Maßangaben |


Zudem werden folgende Informationen aus dem Datensatz ermittelt:

| Attribut | ausgewertetes LIDO-Feld | Beschreibung |
|:---------|:--------------|:-----------------|
| `.license` | `<lido:recordRights/>` | normierte Rechteangabe des Datensatzes |
| `.age` | `<lido:recordMetadataDate/>` | Abstand in Tagen zur letzten Aktualisierung | 


Die Ergebnisse werden in die schon bestehende SQLITE-Datenbank (`berlinerkultuerbedaten.db`) geschrieben.

Es wird eine neue Tabelle (`analysis`) mit folgenden Spalten erstellt:


| Column  | Typ | Beschreibung
|:---------|:---------|:------------|
| OBJID   | Primary Key, VARCHAR(32) | DDB-Objekt-ID (32-stellig), verknüpft die Daten aus `analysis` mit `objects` | 
| FORMAT | VARCHAR(5) | Format des Datensatzes |
| VALID | INTEGER | Gibt an, ob schemavalides LIDO (`1`) oder nicht (`0`) |
| LICENSE  | VARCHAR(40) | möglichst vereinheitlichte Lizenzangabe |
| AGE  | INTEGER | Alter des Datensatzes in Tagen |
| ACTORCOV | REAL | Abdeckungsgrad für Personen/Körperschaften |
| PLACECOV | REAL | Abdeckungsgrad für Orte |
| SUBJECTCOV | REAL | Abdeckungsgrad für Inhaltsschlagwörter |
| OBJTYPECOV | REAL | Abdeckungsgrad für Objekttypen |
| DATECOV | REAL | Anteil standardisierter Datumsangaben |
| MEASURECOV | REAL | Anteil standardisierter Maßangaben |

In [5]:
import sqlite3
from pathlib import Path
from lxml import etree
import requests
from tqdm import tqdm
import json
from DDBhelpers import *

In [2]:
con = sqlite3.connect("berlinerkultuerbedaten.db")
ReadFromObjects = con.cursor()
ReadFromObjects.execute("SELECT OBJID,INSTID FROM objects;")

WriteToAnalysis = con.cursor()
WriteToAnalysis.execute("""CREATE TABLE IF NOT EXISTS analysis
    (
    OBJID VARCHAR(32) PRIMARY KEY,
    VALID INTEGER,
    FORMAT VARCHAR(10),
    LICENSE VARCHAR(10),
    AGE INTEGER,
    OBJTYPECOV REAL,
    ACTORCOV REAL,
    PLACECOV REAL,
    SUBJECTCOV REAL,
    DATECOV REAL,
    MEASURECOV REAL,
    ACTORNUM INTEGER,
    PLACENUM INTEGER,
    SUBJECTNUM INTEGER,
    OBJTYPENUM INTEGER
    );""")

con.commit()

In [3]:
# Pfad, in dem lokal die Metadaten-Dateien liegen
directoryPath = Path("data")

In [6]:
log = []
URIs = {
    'actor' : [],
    'place' : [],
    'subject' : [],
    'objectType' : []
}



for r in tqdm(ReadFromObjects):
    OBJID, inst = r
    path = directoryPath / OBJID
    try:
        tree = etree.parse(path)
        # Format ermitteln
        FORMAT = formatGuesser(tree)
        if FORMAT == "lido":
            # Schemavalidierung
            VALID = LIDOvalidator(tree, xmlschema)

            # LIDO-Instanz erzeugen
            lidoObject = LIDO(tree)

            # URIs einsammeln
            URIs['actor'].append(lidoObject.actorUris)
            URIs['place'].append(lidoObject.placeUris)
            URIs['subject'].append(lidoObject.subjectUris)
            URIs['objectType'].append(lidoObject.objectWorkTypeUris)

            # Abdeckungsmetriken
            LICENSE = lidoObject.license
            AGE = lidoObject.age
            ACTORCOV = lidoObject.actorCoverage
            PLACECOV = lidoObject.placeCoverage
            SUBJECTCOV = lidoObject.subjectCoverage
            OBJTYPECOV = lidoObject.objectWorkTypeCoverage
            DATECOV = lidoObject.dateCoverage
            MEASURECOV = lidoObject.measureCoverage
            WriteToAnalysis.execute("""INSERT OR REPLACE INTO analysis VALUES (
        :OBJID,
        :VALID,
        :FORMAT,
        :LICENSE,
        :AGE,
        :OBJTYPECOV,
        :ACTORCOV,
        :PLACECOV,
        :SUBJECTCOV,
        :DATECOV,
        :MEASURECOV,
        :ACTORNUM,
        :PLACENUM,
        :SUBJECTNUM,
        :OBJTYPENUM
        )
        """, {
            "OBJID" : OBJID,
            "VALID" : VALID,
            "FORMAT" : FORMAT,
            "LICENSE" : LICENSE,
            "AGE" : AGE,
            "OBJTYPECOV" : OBJTYPECOV,
            "ACTORCOV" : ACTORCOV,
            "PLACECOV" : PLACECOV,
            "SUBJECTCOV" : SUBJECTCOV,
            "DATECOV" : DATECOV,
            "MEASURECOV" : MEASURECOV,
            "ACTORNUM" : lidoObject.numActors,
            "PLACENUM" : lidoObject.numPlaces,
            "SUBJECTNUM" : lidoObject.numSubjects,
            "OBJTYPENUM" : lidoObject.numObjectWorkTypes
        })
        else:
            WriteToAnalysis.execute("""INSERT OR REPLACE INTO analysis (OBJID, FORMAT) VALUES (
        :OBJID,
        :FORMAT)
        """, {
    'OBJID' : OBJID,
    'FORMAT' : FORMAT
})      
    except Exception as e:
        print(e)
        log.append((OBJID,e))

con.commit()     
con.close()


with open('URIs.json', 'w') as OUT:
    json.dump(URIs, OUT)

62013it [00:53, 1158.62it/s]


KeyboardInterrupt: 